# *Data Load*

In [1]:
import pandas as pd
from pshmodule.utils import filemanager as fm

In [2]:
df_users = fm.load('../data/df_users.csv')

extension : .csv
Loaded 445220 records from ../data/df_users.csv


In [3]:
df_users.head()

,user_uuid,marketing_channel,sex,age_group,date_joined,os
0,05b0058df377da90c21c585649cbf415,channel_A,male,30-39,2030-05-10,android
1,28b7062943065c84bb902a0c1d018398,channel_B,male,20-29,2030-05-10,android
2,f8c30b791fbe2d6c80585b20fb6fdbe0,channel_A,male,30-39,2030-05-25,android
3,91cd8505d196bea35d48f4f1eacbc106,channel_A,female,30-39,2030-05-20,iOS
4,4b34dfb277abb6ad04dcf6968fe416d0,channel_A,male,30-39,2030-05-20,android


In [4]:
df_user_activities = fm.load('../data/df_user_activities.csv')

extension : .csv
Loaded 405085 records from ../data/df_user_activities.csv


In [5]:
df_user_activities.head()

,user_uuid,visits,revenue
0,479111c2a4a6f4bab457c100c33ac461,5,115.0
1,dfe8227b49199cd5edc20fcd92b78e0b,18,44.0
2,91f1cba54ecba59d0f4a7170ce87492f,6,120.0
3,cf1682c626c9b838bf8bfc9ebdafde76,1,584.0
4,193c6a06b21e8fce199dc508235e5b6b,8,273.0


# 

# *Data Inner Join*

In [6]:
df_test = df_user_activities[df_user_activities.user_uuid.str.contains('05b0058df377da90c21c585649cbf415')]

In [7]:
df_test.head()

,user_uuid,visits,revenue
393704,05b0058df377da90c21c585649cbf415,23,304.0


In [8]:
df = pd.merge(df_users, df_user_activities, left_on='user_uuid', right_on='user_uuid', how='inner')

In [9]:
df.head()

,user_uuid,marketing_channel,sex,age_group,date_joined,os,visits,revenue
0,05b0058df377da90c21c585649cbf415,channel_A,male,30-39,2030-05-10,android,23,304.0
1,28b7062943065c84bb902a0c1d018398,channel_B,male,20-29,2030-05-10,android,5,195.0
2,f8c30b791fbe2d6c80585b20fb6fdbe0,channel_A,male,30-39,2030-05-25,android,1,78.0
3,91cd8505d196bea35d48f4f1eacbc106,channel_A,female,30-39,2030-05-20,iOS,18,202.0
4,4b34dfb277abb6ad04dcf6968fe416d0,channel_A,male,30-39,2030-05-20,android,9,197.0


In [10]:
df.shape

(405085, 8)

# 

# *Encoding*

### marketing_channel
channel_A: 0<br>
channel_B: 1

In [11]:
df['marketing_channel'] = df.marketing_channel.replace('channel_A', 0)
df['marketing_channel'] = df.marketing_channel.replace('channel_B', 1)

### sex
male: 0<br>
female: 1

In [12]:
df['sex'] = df.sex.replace('male', 0)
df['sex'] = df.sex.replace('female', 1)

### age_group
20-29: 0<br>
30-39: 1<br>
40-49: 2<br>
50-59: 3<br>
-99: 4

In [13]:
df.age_group.value_counts()

30-39    158146
20-29    128978
40-49     77151
50-59     28413
-99        4120
Name: age_group, dtype: int64

In [14]:
df = df[~df['age_group'].isnull()]

In [15]:
def age_replace(age):
    if age == '20-29':
        age = 0
    elif age == '30-39':
        age = 1
    elif age == '40-49':
        age = 2
    elif age == '50-59':
        age = 3
    elif age == '-99':
        age = 4
    return int(age)

In [16]:
df['age_group'] = df.age_group.apply(age_replace)

### date_joined
~ 2030-01-31 : 0<br>
2030-02-01 ~ 2030-02-31: 1<br>
2030-03-01 ~ 2030-03-31: 2<br>
2030-04-01 ~ 2030-04-31: 3<br>
2030-05-01 ~ 2030-05-31: 4

In [17]:
def date_replace(date):
    if date <='2030-01-31':
        date = 0
    elif date >= '2030-02-01' and date <='2030-02-31':
        date = 1
    elif date >= '2030-03-01' and date <='2030-03-31':
        date = 2
    elif date >= '2030-04-01' and date <='2030-04-31':
        date = 3
    elif date >= '2030-05-01' and date <='2030-05-31':
        date = 4
    return date

In [18]:
df['date_joined'] = df.date_joined.apply(date_replace)

### os
android: 0<br>
iOS: 1

In [21]:
df['os'] = df.sex.replace('android', 0)
df['os'] = df.sex.replace('iOS', 1)

In [22]:
df.head()

,user_uuid,marketing_channel,sex,age_group,date_joined,os,visits,revenue
0,05b0058df377da90c21c585649cbf415,0,0,1,4,0,23,304.0
1,28b7062943065c84bb902a0c1d018398,1,0,0,4,0,5,195.0
2,f8c30b791fbe2d6c80585b20fb6fdbe0,0,0,1,4,0,1,78.0
3,91cd8505d196bea35d48f4f1eacbc106,0,1,1,4,1,18,202.0
4,4b34dfb277abb6ad04dcf6968fe416d0,0,0,1,4,0,9,197.0


### data check

In [23]:
df.marketing_channel.value_counts()

0    321485
1     75323
Name: marketing_channel, dtype: int64

In [24]:
df.sex.value_counts()

0    214185
1    182623
Name: sex, dtype: int64

In [25]:
df.age_group.value_counts()

1    158146
0    128978
2     77151
3     28413
4      4120
Name: age_group, dtype: int64

In [26]:
df.date_joined.value_counts()

4    361455
3     31502
2      3456
1       346
0        49
Name: date_joined, dtype: int64

In [27]:
df.os.value_counts()

0    214185
1    182623
Name: os, dtype: int64

In [28]:
df.shape

(396808, 8)

# 

# *Data Save*

In [29]:
fm.save('../data/df_users_whole_info.pickle', df)

Saved 396808 records
